In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%pip install transformers

In [3]:
from torch import nn
from SNLIDataset import SNLIDataset
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer, DistilBertTokenizer

C:\Users\nayut\.conda\envs\cr\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
snli = load_dataset("snli")
snli_train = concatenate_datasets([snli["train"], snli["validation"]]).filter(lambda x: x["label"] != -1)
snli_test = snli["test"].filter(lambda x: x["label"] != -1)

Found cached dataset snli (C:/Users/nayut/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 259.29it/s]
Loading cached processed dataset at C:/Users/nayut/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-52fcad8e87d62c80.arrow
Loading cached processed dataset at C:/Users/nayut/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-85328198e48f3415.arrow


In [21]:
TRAINING_REQUIRED_LENGTH=50000
TESTING_REQUIRED_LENGTH=10000
#TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")
TOKENIZER = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
snli_train_dataset = SNLIDataset(snli_train["premise"][:TRAINING_REQUIRED_LENGTH], snli_train["hypothesis"][:TRAINING_REQUIRED_LENGTH],snli_train["label"][:TRAINING_REQUIRED_LENGTH], TOKENIZER,split=False)
snli_test_dataset = SNLIDataset(snli_test["premise"][:TESTING_REQUIRED_LENGTH], snli_test["hypothesis"][:TESTING_REQUIRED_LENGTH],snli_test["label"][:TESTING_REQUIRED_LENGTH], TOKENIZER, split=False)

In [22]:
from train import train_double_encoder, train_single_encoder
from models import BasicDoubleDistilBertClassifier, BasicSingleDistilBertClassifier

In [23]:
snli_test_dataloader = DataLoader(snli_test_dataset, batch_size=16, shuffle=True)
snli_train_dataloader = DataLoader(snli_train_dataset, batch_size=16, shuffle=True)

In [1]:
alpha = 1
delta = 0.5
n_epochs = 10
lr = 0.0001
data_name = "snli"
context = "test1"
report_period = 50
criterion = nn.CrossEntropyLoss()
#adv_model = AdversialClassifier(alpha)
basic_model =  BasicSingleDistilBertClassifier()
optimizer = torch.optim.Adam(basic_model.parameters(), lr=lr)
train_single_encoder(basic_model, snli_train_dataloader, snli_test_dataloader, 
                     optimizer, n_epochs, data_name, context, device, criterion, report_period=report_period)

NameError: name 'nn' is not defined